In [69]:
import numpy as np
import matplotlib.pyplot as plt
import cvxpy as cp
from scipy.io import loadmat

# Intervalo entorno a la media

Importar datos del problema

In [70]:
data = loadmat("Datos_Manufacturing_media.mat")
A_media = data['A_media']
A_hat_media = data['A_hat_media']
b = data['b'].flatten()

c = np.array([1.0,1.0,1.0,1.0,1.0]) 
Gamma = 3
tolerance = 1e-6
max_iter = 30

In [71]:
n_vars = A_media.shape[1]
x = cp.Variable(n_vars)
constraints = [A_media @ x <= b, x >= 0]
objective = cp.Maximize(c @ x)  # ← usar directamente Maximize
history = []
iteration = 0

while iteration < max_iter:
    problem = cp.Problem(objective, constraints)
    problem.solve()

    if x.value is None or problem.status not in ["optimal", "optimal_inaccurate"]:
        print(f"\n❌ Problema infactible o mal definido en la iteración {iteration}")
        break

    x_val = x.value
    f_val = float(c @ x_val)
    history.append(np.copy(x_val))

    print(f"\n🟦 Iteración {iteration}")
    print(f"x* = [{', '.join(f'{xi:.4f}' for xi in x_val)}]")
    print(f"f(x*) = {f_val:.4f}")

    violated = False

    for i in range(len(A)):
        z = A_hat_media[i] * np.abs(x_val)
        top_idx = np.argsort(-z)[:Gamma]

        a_bar = np.copy(A[i])
        a_bar[top_idx] += A_hat_media[i][top_idx] * np.sign(x_val[top_idx])
        lhs = a_bar @ x_val

        if lhs > b[i] + tolerance:
            violated = True
            constraints.append(a_bar @ x <= b[i])

            terms = [f"{a_bar[j]:.4f}·x{j+1}" for j in range(n_vars)]
            expr_str = " + ".join(terms)

            print(f"→ ⚠️ Se añade corte en restricción {i}:")
            print(f"   {expr_str} ≤ {b[i]:.4f}")
            print(f"   Valor actual: {lhs:.4f} > {b[i]:.4f}")
        else:
            print(f"   ✅ Restricción {i} verificada: {lhs:.4f} ≤ {b[i]:.4f}")

    if not violated:
        print("✅ Solución robusta encontrada.")
        break

    iteration += 1

# --- Resultado final ---
print("\n🔷 Resultado final:")
if x.value is not None:
    print("x robusto =", x.value)
    print("Costo objetivo =", c @ x.value)
else:
    print("No se encontró solución robusta.")


🟦 Iteración 0
x* = [0.0000, 0.0000, 0.0000, 0.0000, 6.8367]
f(x*) = 6.8367
→ ⚠️ Se añade corte en restricción 0:
   119.0841·x1 + 64.4444·x2 + 98.3784·x3 + 81.8288·x4 + 92.6937·x5 ≤ 480.0000
   Valor actual: 633.7222 > 480.0000
   ✅ Restricción 1 verificada: 24.8301 ≤ 120.0000
   ✅ Restricción 2 verificada: 61.9022 ≤ 135.0000

🟦 Iteración 1
x* = [0.0000, 5.1386, 0.0000, 0.0000, 1.6058]
f(x*) = 6.7444
→ ⚠️ Se añade corte en restricción 0:
   104.0841·x1 + 74.4444·x2 + 98.3784·x3 + 81.8288·x4 + 92.6937·x5 ≤ 480.0000
   Valor actual: 531.3857 > 480.0000
   ✅ Restricción 1 verificada: 29.5769 ≤ 120.0000
   ✅ Restricción 2 verificada: 47.0327 ≤ 135.0000

🟦 Iteración 2
x* = [0.0000, 6.4478, 0.0000, 0.0000, 0.0000]
f(x*) = 6.4478
   ✅ Restricción 0 verificada: 480.0000 ≤ 480.0000
   ✅ Restricción 1 verificada: 29.7945 ≤ 120.0000
   ✅ Restricción 2 verificada: 40.7717 ≤ 135.0000
✅ Solución robusta encontrada.

🔷 Resultado final:
x robusto = [1.21779238e-10 6.44776119e+00 3.94138614e-10 3.7015

# Intervalo [min max]

In [78]:
data = loadmat("Datos_Manufacturing_MinMax.mat")
A_MinMax = data['A_MinMax']
A_hat_MinMax = data['A_hat_MinMax']
b = data['b'].flatten()

c = np.array([1.0,1.0,1.0,1.0,1.0]) 
Gamma = 3
tolerance = 1e-6
max_iter = 30

In [79]:
n_vars = A_MinMax.shape[1]
x = cp.Variable(n_vars)
constraints = [A_MinMax @ x <= b, x >= 0]
objective = cp.Maximize(c @ x)  # ← usar directamente Maximize
history = []
iteration = 0

while iteration < max_iter:
    problem = cp.Problem(objective, constraints)
    problem.solve()

    if x.value is None or problem.status not in ["optimal", "optimal_inaccurate"]:
        print(f"\n❌ Problema infactible o mal definido en la iteración {iteration}")
        break

    x_val = x.value
    f_val = float(c @ x_val)
    history.append(np.copy(x_val))

    print(f"\n🟦 Iteración {iteration}")
    print(f"x* = [{', '.join(f'{xi:.4f}' for xi in x_val)}]")
    print(f"f(x*) = {f_val:.4f}")

    violated = False

    for i in range(len(A)):
        z = A_hat_MinMax[i] * np.abs(x_val)
        top_idx = np.argsort(-z)[:Gamma]

        a_bar = np.copy(A[i])
        a_bar[top_idx] += A_hat_MinMax[i][top_idx] * np.sign(x_val[top_idx])
        lhs = a_bar @ x_val

        if lhs > b[i] + tolerance:
            violated = True
            constraints.append(a_bar @ x <= b[i])

            terms = [f"{a_bar[j]:.4f}·x{j+1}" for j in range(n_vars)]
            expr_str = " + ".join(terms)

            print(f"→ ⚠️ Se añade corte en restricción {i}:")
            print(f"   {expr_str} ≤ {b[i]:.4f}")
            print(f"   Valor actual: {lhs:.4f} > {b[i]:.4f}")
        else:
            print(f"   ✅ Restricción {i} verificada: {lhs:.4f} ≤ {b[i]:.4f}")

    if not violated:
        print("✅ Solución robusta encontrada.")
        break

    iteration += 1

# --- Resultado final ---
print("\n🔷 Resultado final:")
if x.value is not None:
    print("x robusto =", x.value)
    print("Costo objetivo =", c @ x.value)
else:
    print("No se encontró solución robusta.")


🟦 Iteración 0
x* = [3.3971, 0.0000, 3.4600, 0.0000, 0.0000]
f(x*) = 6.8571
→ ⚠️ Se añade corte en restricción 0:
   154.0841·x1 + 64.4444·x2 + 148.3784·x3 + 123.3288·x4 + 87.6937·x5 ≤ 480.0000
   Valor actual: 1036.8348 > 480.0000
   ✅ Restricción 1 verificada: 31.5788 ≤ 120.0000
   ✅ Restricción 2 verificada: 110.9261 ≤ 135.0000

🟦 Iteración 1
x* = [0.0000, 6.3234, 0.4885, 0.0000, 0.0000]
f(x*) = 6.8120
→ ⚠️ Se añade corte en restricción 0:
   154.0841·x1 + 113.9444·x2 + 148.3784·x3 + 72.8288·x4 + 87.6937·x5 ≤ 480.0000
   Valor actual: 793.0095 > 480.0000
   ✅ Restricción 1 verificada: 42.4364 ≤ 120.0000
   ✅ Restricción 2 verificada: 81.4717 ≤ 135.0000

🟦 Iteración 2
x* = [0.0000, 0.0000, 0.0000, 0.0000, 5.4736]
f(x*) = 5.4736
→ ⚠️ Se añade corte en restricción 0:
   104.0841·x1 + 113.9444·x2 + 98.3784·x3 + 123.3288·x4 + 137.1937·x5 ≤ 480.0000
   Valor actual: 750.9431 > 480.0000
   ✅ Restricción 1 verificada: 30.2245 ≤ 120.0000
   ✅ Restricción 2 verificada: 80.7868 ≤ 135.0000

🟦 I